In [39]:
import os
import pandas as pd
#import re
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
import warnings
import pathlib 
from pathlib import Path 
import threading
from tqdm import tqdm

warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')

# MANUAL input to change
your_ing_id = "XO21BM"
date = datetime.strptime('2024', "%Y")
update_final_overview = True

# DO NOT CHANGE ANYTHING BELOW - only when developing code
# determining operating system
is_windows = os.name == 'nt'
if is_windows:
    base_dir = os.path.join("C:", "\\Users", your_ing_id, "ING")
else:
    base_dir = pathlib.Path(f"/Users/{your_ing_id}/Library/CloudStorage/OneDrive-SharedLibraries-ING")

# Define paths dynamically
directory_path0 = os.getcwd()
directory_path = os.path.join(base_dir, "Product Evaluation and Risk Assessment Library (PEARL) - PEARL_Repository")
directory_path2 = os.path.join(base_dir, "Product Evaluation and Risk Assessment Library (PEARL) - MI Dashboard")
directory_path3 = os.path.join(directory_path0, "Intermediate results")
final_overview = pd.read_csv('final_overview.csv', sep=';')

# Print paths
print(f"Operating System: {'Windows' if is_windows else 'Mac/Linux'}")
print(f"PEARL Repository Path: {directory_path}")
print(f"MI Dashboard Path: {directory_path2}")
print(f"Intermediate Results Path: {directory_path3}")

Operating System: Mac/Linux
PEARL Repository Path: /Users/XO21BM/Library/CloudStorage/OneDrive-SharedLibraries-ING/Product Evaluation and Risk Assessment Library (PEARL) - PEARL_Repository
MI Dashboard Path: /Users/XO21BM/Library/CloudStorage/OneDrive-SharedLibraries-ING/Product Evaluation and Risk Assessment Library (PEARL) - MI Dashboard
Intermediate Results Path: /Users/xo21bm/Documents/NFR/Intermediate results


In [159]:
# Initialize DataFrames
dt1, dt2, dt3, dt4, dt5, dt6, dt7 = [pd.DataFrame() for _ in range(7)]

# Check if directory is empty
all_files = os.listdir(directory_path3)
if not all_files:
    raise ValueError("Error: directory_path3 is empty!")

# Process files
for f in tqdm(all_files):
    file_path = os.path.join(directory_path3, f)

    # Ensure it's a file, not a directory
    if os.path.isfile(file_path):
        try:
            dt_f = pd.read_csv(file_path, sep=';', encoding='utf-8')
            if 'process_module_selection' in f:
                dt1 = pd.concat([dt1, dt_f])
            elif 'risk_summary_approval' in f:
                dt2 = pd.concat([dt2, dt_f])
            elif 'general_risk_ident_1' in f:
                dt3 = pd.concat([dt3, dt_f])
            elif 'general_risk_ident_2' in f:
                dt4 = pd.concat([dt4, dt_f])
            elif 'module_selected' in f:
                dt5 = pd.concat([dt5, dt_f])
            elif 'assesment_stage' in f:
                dt6 = pd.concat([dt6, dt_f])
            elif 'journey_summary' in f:
                dt7 = pd.concat([dt7, dt_f])

        except Exception as e:
            print(f"Error processing file {f}: {e}")

# Create backup copies before modification
dataframes = [dt1, dt2, dt3, dt4, dt5, dt6, dt7]
dataframes_backup = [df.copy() for df in dataframes]

# Normalize folder paths (Mac-safe)
for i in range(len(dataframes)):
    if 'Folder' in dataframes[i].columns:
        dataframes[i]['Folder'] = dataframes[i]['Folder'].apply(lambda x: str(Path(x)))

100%|██████████████████████████████████████| 5403/5403 [00:17<00:00, 310.74it/s]


In [160]:
# merge with final overview each 
dt1 = dt1.merge(final_overview, on="Folder", how="left")
dt2 = dt2.merge(final_overview, on="Folder", how="left")
dt3 = dt3.merge(final_overview, on="Folder", how="left")
dt4 = dt4.merge(final_overview, on="Folder", how="left")
dt5 = dt5.merge(final_overview, on="Folder", how="left")

In [202]:
### data manipulation for each individual df

cond = dt1['File'].isna()
dt1.loc[cond, "Missing data"] = True
dt1.loc[~cond, "Missing data"] = False

cond = (dt1['ORM_check'] == 'ORM') & (dt1['IRM_check'] == 'IRM')
dt1.loc[cond, "Data correct"] = True  
dt1.loc[~cond, "Data correct"] = False  

cond = (dt1['IRM'] == 'Yes')
dt1.loc[cond, 'IRM'] = '1'
dt1.loc[~cond, 'IRM'] = '0'

dt1['IRM'] = pd.to_numeric(dt1['IRM'], errors='coerce')

cond = (dt1['ORM'] == 'Yes')
dt1.loc[cond, 'ORM'] = '1'
dt1.loc[~cond, 'ORM'] = '0'

dt1['ORM'] = pd.to_numeric(dt1['ORM'], errors='coerce')

In [203]:
# IRM & ORM dt2 risk summary approval
cond = dt2['File'].isna()
dt2.loc[cond, 'Missing data'] = True
dt2.loc[~cond, 'Missing data'] = False

cond = (dt2['ORM_check'] == 'ORM') & (dt2['IRM_check'] == 'IRM')
dt2.loc[cond, 'Data correct'] = True
dt2.loc[~cond, 'Data correct'] = False

cond = (dt2['ORM_invited_or_challange'] == 'To be invited for challenge')
dt2.loc[cond, 'ORM_invited_or_challange'] = '1'
dt2.loc[~cond, 'ORM_invited_or_challange'] = '0'

dt2['ORM_invited_or_challange'] = pd.to_numeric(dt2['ORM_invited_or_challange'], errors='coerce')

cond = (dt2['IRM_invited_or_challange'] == 'To be informed')
dt2.loc[cond, 'IRM_invited_or_challange'] = '1'
dt2.loc[~cond, 'IRM_invited_or_challange'] = '0'

dt2['IRM_invited_or_challange'] = pd.to_numeric(dt2['IRM_invited_or_challange'], errors='coerce')

In [204]:
# 
cond = dt3['File'].isna()
dt3.loc[cond, 'Missing data'] = True
dt3.loc[~cond, 'Missing data'] = False

cond = (dt3['ORM_check'] == 'ORM') & (dt3['IRM_check'] == 'IRM')
dt3.loc[cond, 'Data correct'] = True
dt3.loc[~cond, 'Data correct'] = False

cond = (dt3['ORM_invited'] == 'Yes')
dt3.loc[cond, 'ORM_invited'] = '1'
dt3.loc[~cond, 'ORM_invited'] = '0'

dt3['ORM_invited'] = pd.to_numeric(dt3['ORM_invited'], errors='coerce')

cond = (dt3['IRM_invited'] == 'Yes')
dt3.loc[cond, 'IRM_invited'] = '1'
dt3.loc[~cond, 'IRM_invited'] = '0'

dt3['IRM_invited'] = pd.to_numeric(dt3['IRM_invited'], errors='coerce')

cond = (dt3['ORM_part_of_risk_asses'] == 'Yes')
dt3.loc[cond, 'ORM_part_of_risk_asses'] = '1'
dt3.loc[~cond, 'ORM_part_of_risk_asses'] = '0'

dt3['ORM_part_of_risk_asses'] = pd.to_numeric(dt3['ORM_part_of_risk_asses'], errors='coerce')

cond = (dt3['IRM_part_of_risk_asses'] == 'Yes')
dt3.loc[cond, 'IRM_part_of_risk_asses'] = '1'
dt3.loc[~cond, 'IRM_part_of_risk_asses'] = '0'

dt3['IRM_part_of_risk_asses'] = pd.to_numeric(dt3['IRM_part_of_risk_asses'], errors='coerce')

In [205]:
#
cond = dt4['File'].isna()
dt4.loc[cond, 'Missing data'] = True
dt4.loc[~cond, 'Missing data'] = False

cond = (dt4['ORM_check'] == 'ORM') & (dt4['IRM_check'] == 'IRM')
dt4.loc[cond, 'Data correct'] = True
dt4.loc[~cond, 'Data correct'] = False

cond = (dt4['ORM_person'] == 'Not applicable')
dt4.loc[cond, 'ORM_person'] = '0'
dt4.loc[~cond, 'ORM_person'] = '1'

dt4['ORM_person'] = pd.to_numeric(dt4['ORM_person'], errors='coerce')

cond = (dt4['IRM_person'] == 'Not applicable')
dt4.loc[cond, 'IRM_person'] = '0'
dt4.loc[~cond, 'IRM_person'] = '1'

dt4['IRM_person'] = pd.to_numeric(dt4['IRM_person'], errors='coerce')

cond = (dt4['ORM_opinion'] == 'Not applicable')
dt4.loc[cond, 'ORM_opinion'] = '0'
dt4.loc[~cond, 'ORM_opinion'] = '1'

dt4['ORM_opinion'] = pd.to_numeric(dt4['ORM_opinion'], errors='coerce')

cond = (dt4['IRM_opinion'] == 'Not applicable')
dt4.loc[cond, 'IRM_opinion'] = '0'
dt4.loc[~cond, 'IRM_opinion'] = '1'

dt4['IRM_opinion'] = pd.to_numeric(dt4['IRM_opinion'], errors='coerce')

cond = (dt4['ORM_challenge'] == 'Not applicable')
dt4.loc[cond, 'ORM_challenge'] = '0'
dt4.loc[~cond, 'ORM_challenge'] = '1'

dt4['ORM_challenge'] = pd.to_numeric(dt4['ORM_challenge'], errors='coerce')

cond = (dt4['IRM_challenge'] == 'Not applicable')
dt4.loc[cond, 'IRM_challenge'] = '0'
dt4.loc[~cond, 'IRM_challenge'] = '1'

dt4['IRM_challenge'] = pd.to_numeric(dt4['IRM_challenge'], errors='coerce')

In [220]:
### first data analysis

# dt1
dt1["End Date"] = pd.to_datetime(dt1["End Date"])  # Convert to datetime
dt1["Year"] = dt1["End Date"].dt.year
dt1["Quarter"] = dt1["End Date"].dt.to_period("Q")
dt1["Quarter"] = dt1["Quarter"].fillna("Unknown").astype(str)
dt1["Quarter"] = dt1["End Date"].dt.to_period("Q").astype(str)

dt1_grouped = dt1.groupby(['Missing data', 
                           'Data correct', 
                           'Quarter']).agg({'ORM': 'sum', 
                                       'IRM': 'sum', 
                                       'Folder': 'count'})


In [221]:
dt1_grouped

ORM  IRM  Folder
Missing data Data correct Quarter                  
False        False        2024Q4     2    0       2
                          NaT        3    1       3
             True         2024Q1    55   22     126
                          2024Q2    53   26     123
                          2024Q3    52   24     138
                          2024Q4    90   40     220
                          2025Q1    43   28     151
                          2025Q2    29   21      93
                          NaT       57   74     238

In [226]:
# create percentages
dt1_percent = dt1_grouped.copy()

# Divide ORM & IRM by total Folder count per quarter
cols_to_percent = ["ORM", "IRM"]
dt1_percent[cols_to_percent] = dt1_percent[cols_to_percent].div(
    dt1_percent["Folder"], axis=0) * 100
dt1_percent[cols_to_percent] = dt1_percent[cols_to_percent].round(2)


In [227]:
dt1_percent

ORM    IRM  Folder
Missing data Data correct Quarter                       
False        False        2024Q4   100.00   0.00       2
                          NaT      100.00  33.33       3
             True         2024Q1    43.65  17.46     126
                          2024Q2    43.09  21.14     123
                          2024Q3    37.68  17.39     138
                          2024Q4    40.91  18.18     220
                          2025Q1    28.48  18.54     151
                          2025Q2    31.18  22.58      93
                          NaT       23.95  31.09     238

In [218]:
### first data analysis

# dt2
dt2["End Date"] = pd.to_datetime(dt2["End Date"])  # Convert to datetime
dt2["Year"] = dt2["End Date"].dt.year
dt2["Quarter"] = dt2["End Date"].dt.to_period("Q")
dt2["Quarter"] = dt2["Quarter"].fillna("Unknown").astype(str)
dt2["Quarter"] = dt2["End Date"].dt.to_period("Q").astype(str)

dt2_grouped = dt2.groupby(['Missing data', 
                           'Data correct', 
                           'Quarter']).agg({'ORM_invited_or_challange': 'sum', 
                                       'IRM_invited_or_challange': 'sum', 
                                       'Folder': 'count'})

In [219]:
dt2_grouped

ORM_invited_or_challange  \
Missing data Data correct Quarter                             
False        True         2024Q1                          5   
                          2024Q2                          3   
                          2024Q3                          5   
                          2024Q4                         22   
                          2025Q1                         17   
                          2025Q2                         31   
                          NaT                            35   

                                   IRM_invited_or_challange  Folder  
Missing data Data correct Quarter                                    
False        True         2024Q1                          3       5  
                          2024Q2                          2       3  
                          2024Q3                          5       5  
                          2024Q4                         12      22  
                          2025Q1                          4      17  
                          2025Q2                          1      31  
                          NaT                            10      35

In [228]:
# create percentages
dt2_percent = dt2_grouped.copy()

# Divide ORM & IRM by total Folder count per quarter
cols_to_percent = ["ORM_invited_or_challange", "IRM_invited_or_challange"]
dt2_percent[cols_to_percent] = dt2_percent[cols_to_percent].div(
    dt2_percent["Folder"], axis=0) * 100
dt2_percent[cols_to_percent] = dt2_percent[cols_to_percent].round(2)


In [229]:
dt2_percent

ORM_invited_or_challange  \
Missing data Data correct Quarter                             
False        True         2024Q1                      100.0   
                          2024Q2                      100.0   
                          2024Q3                      100.0   
                          2024Q4                      100.0   
                          2025Q1                      100.0   
                          2025Q2                      100.0   
                          NaT                         100.0   

                                   IRM_invited_or_challange  Folder  
Missing data Data correct Quarter                                    
False        True         2024Q1                      60.00       5  
                          2024Q2                      66.67       3  
                          2024Q3                     100.00       5  
                          2024Q4                      54.55      22  
                          2025Q1                      23.53      17  
                          2025Q2                       3.23      31  
                          NaT                         28.57      35

In [216]:
### first data analysis

# dt3
dt3["End Date"] = pd.to_datetime(dt3["End Date"])  # Convert to datetime
dt3["Year"] = dt3["End Date"].dt.year

dt3["Quarter"] = dt3["End Date"].dt.to_period("Q")
dt3["Quarter"] = dt3["Quarter"].fillna("Unknown").astype(str)
dt3["Quarter"] = dt3["End Date"].dt.to_period("Q").astype(str)

dt3_grouped = dt3.groupby(['Missing data', 
                           'Data correct', 
                           'Quarter']).agg({'ORM_invited': 'sum', 
                                         'IRM_invited': 'sum',    
                                         'ORM_part_of_risk_asses': 'sum', 
                                         'IRM_part_of_risk_asses': 'sum', 
                                         'Folder': 'count'})

In [217]:
dt3_grouped

ORM_invited  IRM_invited  \
Missing data Data correct Quarter                             
False        False        2024Q1             1            1   
                          2024Q2             0            1   
                          2024Q3             3            1   
                          2024Q4             3            3   
                          2025Q1             1            2   
                          2025Q2             1            1   
                          NaT                5            5   
             True         2024Q1            54           21   
                          2024Q2            53           26   
                          2024Q3            52           21   
                          2024Q4            80           41   
                          2025Q1            35           28   
                          2025Q2            18           19   
                          NaT               39           71   

                                   ORM_part_of_risk_asses  \
Missing data Data correct Quarter                           
False        False        2024Q1                        1   
                          2024Q2                        0   
                          2024Q3                        2   
                          2024Q4                        1   
                          2025Q1                        1   
                          2025Q2                        1   
                          NaT                           2   
             True         2024Q1                       29   
                          2024Q2                       29   
                          2024Q3                       22   
                          2024Q4                       61   
                          2025Q1                       19   
                          2025Q2                       15   
                          NaT                          31   

                                   IRM_part_of_risk_asses  Folder  
Missing data Data correct Quarter                                  
False        False        2024Q1                        0       2  
                          2024Q2                        0       2  
                          2024Q3                        1       3  
                          2024Q4                        1       3  
                          2025Q1                        1       2  
                          2025Q2                        1       1  
                          NaT                           2       8  
             True         2024Q1                       15     124  
                          2024Q2                       18     121  
                          2024Q3                       11     135  
                          2024Q4                       29     219  
                          2025Q1                       13     149  
                          2025Q2                        8      92  
                          NaT                           8     231

In [230]:
# create percentages
dt3_percent = dt3_grouped.copy()

# Divide ORM & IRM by total Folder count per quarter
cols_to_percent = ["ORM_invited", "IRM_invited", "ORM_part_of_risk_asses", "IRM_part_of_risk_asses"]
dt3_percent[cols_to_percent] = dt3_percent[cols_to_percent].div(
    dt3_percent["Folder"], axis=0) * 100
dt3_percent[cols_to_percent] = dt3_percent[cols_to_percent].round(2)


In [231]:
dt3_percent

ORM_invited  IRM_invited  \
Missing data Data correct Quarter                             
False        False        2024Q1         50.00        50.00   
                          2024Q2          0.00        50.00   
                          2024Q3        100.00        33.33   
                          2024Q4        100.00       100.00   
                          2025Q1         50.00       100.00   
                          2025Q2        100.00       100.00   
                          NaT            62.50        62.50   
             True         2024Q1         43.55        16.94   
                          2024Q2         43.80        21.49   
                          2024Q3         38.52        15.56   
                          2024Q4         36.53        18.72   
                          2025Q1         23.49        18.79   
                          2025Q2         19.57        20.65   
                          NaT            16.88        30.74   

                                   ORM_part_of_risk_asses  \
Missing data Data correct Quarter                           
False        False        2024Q1                    50.00   
                          2024Q2                     0.00   
                          2024Q3                    66.67   
                          2024Q4                    33.33   
                          2025Q1                    50.00   
                          2025Q2                   100.00   
                          NaT                       25.00   
             True         2024Q1                    23.39   
                          2024Q2                    23.97   
                          2024Q3                    16.30   
                          2024Q4                    27.85   
                          2025Q1                    12.75   
                          2025Q2                    16.30   
                          NaT                       13.42   

                                   IRM_part_of_risk_asses  Folder  
Missing data Data correct Quarter                                  
False        False        2024Q1                     0.00       2  
                          2024Q2                     0.00       2  
                          2024Q3                    33.33       3  
                          2024Q4                    33.33       3  
                          2025Q1                    50.00       2  
                          2025Q2                   100.00       1  
                          NaT                       25.00       8  
             True         2024Q1                    12.10     124  
                          2024Q2                    14.88     121  
                          2024Q3                     8.15     135  
                          2024Q4                    13.24     219  
                          2025Q1                     8.72     149  
                          2025Q2                     8.70      92  
                          NaT                        3.46     231

In [212]:
### first data analysis

# dt4
dt4["End Date"] = pd.to_datetime(dt4["End Date"])  # Convert to datetime
dt4["Year"] = dt4["End Date"].dt.year

dt4["Quarter"] = dt4["End Date"].dt.to_period("Q")
dt4["Quarter"] = dt4["Quarter"].fillna("Unknown").astype(str)
dt4["Quarter"] = dt4["End Date"].dt.to_period("Q").astype(str)

dt4_grouped = dt4.groupby(['Missing data', 
                           'Data correct', 
                           'Quarter']).agg({'ORM_person': 'sum', 
                                         'IRM_person': 'sum',    
                                         'ORM_opinion': 'sum', 
                                         'IRM_opinion': 'sum',
                                         'ORM_challenge': 'sum', 
                                         'IRM_challenge': 'sum', 
                                         'Folder': 'count'})

In [213]:
dt4_grouped

ORM_person  IRM_person  ORM_opinion  \
Missing data Data correct Quarter                                        
False        True         2024Q1           22          11           22   
                          2024Q2           22          13           24   
                          2024Q3           19           9           19   
                          2024Q4           52          24           52   
                          2025Q1           19          12           18   
                          2025Q2           10           5           13   
                          NaT              21           8           32   

                                   IRM_opinion  ORM_challenge  IRM_challenge  \
Missing data Data correct Quarter                                              
False        True         2024Q1            11            107            107   
                          2024Q2            13            105            105   
                          2024Q3             9            113            113   
                          2024Q4            23            109            108   
                          2025Q1            13             44             41   
                          2025Q2             6             18             12   
                          NaT                8             34             10   

                                   Folder  
Missing data Data correct Quarter          
False        True         2024Q1      107  
                          2024Q2      105  
                          2024Q3      120  
                          2024Q4      207  
                          2025Q1      147  
                          2025Q2       87  
                          NaT         214

In [233]:
# create percentages
dt4_percent = dt4_grouped.copy()

# Divide ORM & IRM by total Folder count per quarter
cols_to_percent = ["ORM_person", "IRM_person", "ORM_opinion", "IRM_opinion", "ORM_challenge", "IRM_challenge"]
dt4_percent[cols_to_percent] = dt4_percent[cols_to_percent].div(
    dt4_percent["Folder"], axis=0) * 100
dt4_percent[cols_to_percent] = dt4_percent[cols_to_percent].round(2)

In [234]:
dt4_percent

ORM_person  IRM_person  ORM_opinion  \
Missing data Data correct Quarter                                        
False        True         2024Q1        20.56       10.28        20.56   
                          2024Q2        20.95       12.38        22.86   
                          2024Q3        15.83        7.50        15.83   
                          2024Q4        25.12       11.59        25.12   
                          2025Q1        12.93        8.16        12.24   
                          2025Q2        11.49        5.75        14.94   
                          NaT            9.81        3.74        14.95   

                                   IRM_opinion  ORM_challenge  IRM_challenge  \
Missing data Data correct Quarter                                              
False        True         2024Q1         10.28         100.00         100.00   
                          2024Q2         12.38         100.00         100.00   
                          2024Q3          7.50          94.17          94.17   
                          2024Q4         11.11          52.66          52.17   
                          2025Q1          8.84          29.93          27.89   
                          2025Q2          6.90          20.69          13.79   
                          NaT             3.74          15.89           4.67   

                                   Folder  
Missing data Data correct Quarter          
False        True         2024Q1      107  
                          2024Q2      105  
                          2024Q3      120  
                          2024Q4      207  
                          2025Q1      147  
                          2025Q2       87  
                          NaT         214

In [192]:
### recoding the ones which need it recoding before merging with final overview

dt5["End Date"] = pd.to_datetime(dt5["End Date"], errors="coerce")  
dt5["Year"] = dt5["End Date"].dt.year  
dt5["Quarter"] = dt5["End Date"].dt.to_period("Q")
dt5["Quarter"] = dt5["Quarter"].fillna("Unknown").astype(str)
dt5["Quarter"] = dt5["End Date"].dt.to_period("Q").astype(str)

df_5_module_selected =dt5.groupby(["Folder", 
                                    "File",
                                    "Quarter"]).agg({
    "Applicable": lambda x: x.sum()  
}).reset_index()

df_5_module_selected.rename(columns={"Applicable": "Applicable_modules_selected"}, inplace=True)
df_5_module_selected["Quarter"] = df_5_module_selected["Quarter"].astype(str)


### dt7 -> needs to do the same with this one


In [197]:
### first data analysis


df_wide = df_5_module_selected.pivot_table(index="Quarter", 
                                           columns="Applicable_modules_selected", 
                                           values="File", 
                                           aggfunc="count").fillna(0)

# Reset index to make it cleaner
df_wide.reset_index(inplace=True)


In [199]:
df_wide

Applicable_modules_selected,Quarter,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2024Q1,25.0,58.0,17.0,7.0,3.0,5.0,3.0,3.0,1.0,0.0,0.0,1.0,1.0
1,2024Q2,16.0,63.0,10.0,8.0,7.0,7.0,7.0,0.0,2.0,0.0,0.0,0.0,0.0
2,2024Q3,6.0,97.0,9.0,5.0,4.0,7.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2024Q4,17.0,123.0,14.0,13.0,16.0,12.0,4.0,3.0,5.0,4.0,2.0,3.0,1.0
4,2025Q1,16.0,93.0,11.0,9.0,1.0,7.0,5.0,4.0,3.0,0.0,0.0,0.0,0.0
5,2025Q2,6.0,58.0,10.0,5.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,0.0


In [235]:
df_percent = df_wide.copy()  # Preserve the original DataFrame

# Identify module columns (excluding "Quarter")
module_cols = df_percent.columns.difference(["Quarter"])

# Convert each module count into a percentage relative to the total per quarter
df_percent[module_cols] = df_percent[module_cols].div(df_percent[module_cols].sum(axis=1), axis=0) * 100

# Round percentages for cleaner formatting
df_percent[module_cols] = df_percent[module_cols].round(2)


In [236]:
df_percent

Applicable_modules_selected,Quarter,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2024Q1,20.16,46.77,13.71,5.65,2.42,4.03,2.42,2.42,0.81,0.00,0.00,0.81,0.81
1,2024Q2,13.33,52.50,8.33,6.67,5.83,5.83,5.83,0.00,1.67,0.00,0.00,0.00,0.00
2,2024Q3,4.44,71.85,6.67,3.70,2.96,5.19,4.44,0.74,0.00,0.00,0.00,0.00,0.00
3,2024Q4,7.83,56.68,6.45,5.99,7.37,5.53,1.84,1.38,2.30,1.84,0.92,1.38,0.46
4,2025Q1,10.74,62.42,7.38,6.04,0.67,4.70,3.36,2.68,2.01,0.00,0.00,0.00,0.00
5,2025Q2,6.59,63.74,10.99,5.49,2.20,1.10,2.20,3.30,1.10,1.10,1.10,1.10,0.00
